In [26]:
import cosmos  # type: ignore
from cosmos.execution_types import TRAINING_MODEL  # type: ignore

In [27]:
cosmos.initialization(host="alogin2.bsc.es")

[cosmos.initialization] Creating SSH connection to alogin2.bsc.es
[cosmos.initialization] Connection established and remote path '/gpfs/projects/bsc14/executions' verified.


In [ ]:
# Data parameters
MODEL_CHECKPOINT = "/gpfs/projects/bsc14/abecerr1/hub/models--ufal--robeczech-base/snapshots/98c2170e7b81703dce391ed4c43ac31460a80b1c"
INPUT_PATH = "/gpfs/projects/bsc14/storage/huggingface/cz"
DATA_PATH = f"/gpfs/projects/bsc14/storage/huggingface/{INPUT_PATH.split('/')[-1]}"

INIT_SHARD = 1
SHARDS_QUANTITY = 25
MODEL_NAME = "CardioBerta_CZ"
OPTIM = ""

# MN5 parameters
NODES = 8
GPUS = 4

In [40]:
model_checkpoint_path = MODEL_CHECKPOINT

for i in range(INIT_SHARD, SHARDS_QUANTITY):
    
    data_shard_path = f"{DATA_PATH}/shard_{i}"
    output_dir_path = f"/gpfs/projects/bsc14/storage/models/{MODEL_NAME}/{MODEL_NAME}_shard_{i}"

    job = cosmos.run(
            module_path="torch_cpt.training_cz",
            function_name="continual_pretraining",
            queue="acc_debug",  # acc_debug or acc_bscls
            user="bsc14",
            kwargs={
                "model_checkpoint": model_checkpoint_path,
                "data_path": data_shard_path,
                "output_dir": output_dir_path,
                "column": "text",
            },
            module_purge=True,
            modules=[
                "singularity",
            ],
            time="01:59:59",
            tasks=1,
            nodes=NODES,
            cpus=80,
            gpus=GPUS,
            singularity_path="/gpfs/apps/MN5/ACC/SINGULARITY/SRC/images/nvidiaPytorch24.07_bis.sif",
            custom_command=[
                'export WANDB_DISABLED="1"',
                'export HF_HOME=/gpfs/projects/bsc14/parancibia/',
                'export TORCH_NCCL_ASYNC_ERROR_HANDLING=1',
                'export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True',
                'MASTER_ADDR=$(scontrol show hostnames $SLURM_JOB_NODELIST | head -n 1)',
                'MASTER_PORT=6000',
                'export ACCELERATE_MIXED_PRECISION="no"',
                f'torchrun --nproc_per_node {GPUS} --nnodes $SLURM_NNODES --node_rank $SLURM_PROCID --rdzv_endpoint $MASTER_ADDR:$MASTER_PORT --rdzv_backend c10d --max_restarts 0 --tee 3 ',
            ],
            execution_type=TRAINING_MODEL,
            delete_files_after_execution=False,
            job_exclusive=True,
            watch=False,
        )
    
    # Change the Model checkpoint
    model_checkpoint_path = output_dir_path
    print(job)


[cosmos.run] Preparing configuration to execute 'continual_pretraining'
[cosmos.run] Output remote logs: /gpfs/projects/bsc14/executions/job_20250404_095057/job_20250404_095057.out
[cosmos.run] Error remote logs: /gpfs/projects/bsc14/executions/job_20250404_095057/job_20250404_095057.err
[cosmos.run] Submitted batch job 18624017
[cosmos.run] Job job_20250404_095057 (ID: 18624017) sent.
{'job_id': '18624017', 'job_name': 'job_20250404_095057', 'remote_job_dir': '/gpfs/projects/bsc14/executions/job_20250404_095057', 'out_file': '/gpfs/projects/bsc14/executions/job_20250404_095057/job_20250404_095057.out', 'err_file': '/gpfs/projects/bsc14/executions/job_20250404_095057/job_20250404_095057.err', 'outputs': []}
[cosmos.run] Preparing configuration to execute 'continual_pretraining'
[cosmos.run] Output remote logs: /gpfs/projects/bsc14/executions/job_20250404_095101/job_20250404_095101.out
[cosmos.run] Error remote logs: /gpfs/projects/bsc14/executions/job_20250404_095101/job_20250404_09510